### Import Libraries

In [25]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

### Load Data

In [2]:
df = pd.read_csv('./Assets/Datasets/Telco_Customer.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 16 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Customer ID                   7043 non-null   int64  
 1   Tenure Months                 7043 non-null   int64  
 2   Location                      7043 non-null   object 
 3   Device Class                  7043 non-null   object 
 4   Games Product                 7043 non-null   object 
 5   Music Product                 7043 non-null   object 
 6   Education Product             7043 non-null   object 
 7   Call Center                   7043 non-null   object 
 8   Video Product                 7043 non-null   object 
 9   Use MyApp                     7043 non-null   object 
 10  Payment Method                7043 non-null   object 
 11  Monthly Purchase (Thou. IDR)  7043 non-null   float64
 12  Churn Label                   7043 non-null   object 
 13  Lon

In [4]:
df.shape

(7043, 16)

In [5]:
df.describe()

,Customer ID,Tenure Months,Monthly Purchase (Thou. IDR),Longitude,Latitude,CLTV (Predicted Thou. IDR)
count,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000
mean,3521.000000,32.371149,84.190200,107.043249,-6.404182,5720.384481
std,2033.283305,24.559481,39.117061,0.358314,0.322896,1537.974298
min,0.000000,0.000000,23.725000,106.816670,-6.914740,2603.900000
25%,1760.500000,9.000000,46.150000,106.816670,-6.914740,4509.700000
50%,3521.000000,29.000000,91.455000,106.816670,-6.200000,5885.100000
75%,5281.500000,55.000000,116.805000,107.609810,-6.200000,6994.650000
max,7042.000000,72.000000,154.375000,107.609810,-6.200000,8450.000000


#### Dropping Unnecessary Column

In [6]:
df = df.drop(['Customer ID', 'Longitude', 'Latitude'], axis=1)

In [7]:
df.head(3)

,Tenure Months,Location,Device Class,Games Product,Music Product,Education Product,Call Center,Video Product,Use MyApp,Payment Method,Monthly Purchase (Thou. IDR),Churn Label,CLTV (Predicted Thou. IDR)
0,2,Jakarta,Mid End,Yes,Yes,No,No,No,No,Digital Wallet,70.005,Yes,4210.7
1,2,Jakarta,High End,No,No,No,No,No,No,Pulsa,91.910,Yes,3511.3
2,8,Jakarta,High End,No,No,Yes,No,Yes,Yes,Pulsa,129.545,Yes,6983.6


#### Checking the unique values 

In [13]:
columns = ['Location', 'Games Product', 'Music Product', 'Education Product', 'Call Center', 'Video Product', 'Use MyApp', 'Payment Method', 'Churn Label']

for column in columns:
    print(df[column].unique())

['Jakarta' 'Bandung']
['Yes' 'No' 'No internet service']
['Yes' 'No' 'No internet service']
['No' 'Yes' 'No internet service']
['No' 'Yes']
['No' 'Yes' 'No internet service']
['No' 'Yes' 'No internet service']
['Digital Wallet' 'Pulsa' 'Debit' 'Credit']
['Yes' 'No']


In [14]:
columns_replace_val = ['Games Product', 'Music Product', 'Education Product', 'Video Product', 'Use MyApp']

for column in columns_replace_val:
    df[column] = df[column].replace(['No internet service'], 'No')

#### Encoding

In [30]:
columns=['Games Product', 'Music Product', 'Education Product', 'Video Product', 'Use MyApp', 'Call Center', 'Churn Label', 'Location', 'Device Class', 'Payment Method']

le = LabelEncoder()

for column in columns:
    df[column] = le.fit_transform(df[column])

In [31]:
df

,Tenure Months,Location,Device Class,Games Product,Music Product,Education Product,Call Center,Video Product,Use MyApp,Payment Method,Monthly Purchase (Thou. IDR),Churn Label,CLTV (Predicted Thou. IDR)
0,2,1,2,1,1,0,0,0,0,2,70.005,1,4210.7
1,2,1,0,0,0,0,0,0,0,3,91.910,1,3511.3
2,8,1,0,0,0,1,0,1,1,3,129.545,1,6983.6
3,28,1,0,0,0,1,1,1,1,3,136.240,1,6503.9
4,49,1,0,0,1,1,0,1,1,1,134.810,1,6942.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,72,1,1,0,0,0,0,0,0,1,27.495,0,6897.8
7039,24,1,2,1,0,1,1,1,1,2,110.240,0,2782.0
7040,72,1,0,0,1,1,0,1,1,0,134.160,0,7228.0
7041,11,1,2,1,0,0,0,0,0,3,38.480,0,3630.9
